In [1]:
# ---------------------------------------------
# StrengthLevel Workouts Fetch + Display Script
# ---------------------------------------------

import requests
import re
import json
import pandas as pd
# import matplotlib.pyplot as plt
from IPython.display import display, JSON

# ---------------------------------------------
# 1. Setup
# ---------------------------------------------
USERNAME = "adomasgaudi"
HEADERS = {"User-Agent": "Mozilla/5.0"}
URL = f"https://my.strengthlevel.com/{USERNAME}/workouts"

# ---------------------------------------------
# 2. Get the webpage HTML
# ---------------------------------------------
page = requests.get(URL, headers=HEADERS)
page.raise_for_status()
html_text = page.text

# ---------------------------------------------
# 3. Find the embedded JSON ("window.prefill = [...]")
# ---------------------------------------------
pattern = r"window\.prefill\s*=\s*(\[[\s\S]*?\]);"
match = re.search(pattern, html_text)
if not match:
    raise ValueError("Couldn't find 'window.prefill' in page HTML")

json_text = match.group(1)
prefill = json.loads(json_text)

# ---------------------------------------------
# 4. Extract user_id
# ---------------------------------------------
user_id = prefill[0]["request"]["params"]["user_id"]
print("✅ User ID:", user_id)

# ---------------------------------------------
# 5. Fetch workouts from the API
# ---------------------------------------------
API_URL = "https://my.strengthlevel.com/api/workouts"
params = {"user_id": user_id, "limit": 20}
response = requests.get(API_URL, params=params, headers=HEADERS)
response.raise_for_status()
workouts = response.json()

print(f"✅ Got {len(workouts['data'])} workouts")

# ---------------------------------------------
# 6. Display JSON (expandable view)
# ---------------------------------------------
display(JSON(workouts))

# ---------------------------------------------
# 7. Flatten workouts into a table
# ---------------------------------------------
rows = []
for w in workouts["data"]:
    date = w["date"]
    for e in w["exercises"]:
        name = e["exercise_name"]
        for s in e["sets"]:
            rows.append({
                "date": date,
                "exercise": name,
                "weight": s.get("weight"),
                "reps": s.get("reps"),
                "notes": s.get("notes"),
            })

df = pd.DataFrame(rows)
print("✅ Flattened rows:", len(df))
display(df.head(20))
#
# # ---------------------------------------------
# # 8. Save data locally (optional)
# # ---------------------------------------------
# df.to_csv("workouts_table.csv", index=False)
# with open("workouts_raw.json", "w", encoding="utf-8") as f:
#     json.dump(workouts, f, indent=2)
# print("💾 Saved workouts_table.csv and workouts_raw.json")
#
# # ---------------------------------------------
# # 9. Plot example: Bench Press progress
# # ---------------------------------------------
# plt.figure(figsize=(8, 4))
# bench = df[df["exercise"].str.lower().str.contains("bench")]
# plt.plot(bench["date"], bench["weight"], marker="o")
# plt.title("Bench Press Progress")
# plt.xlabel("Date")
# plt.ylabel("Weight (kg)")
# plt.xticks(rotation=45)
# plt.grid(True)
# plt.tight_layout()
# plt.show()


✅ User ID: 018a46b5-52f8-73e4-91a1-e90941b0eb34
✅ Got 20 workouts


<IPython.core.display.JSON object>